Drive Link for sample data: https://drive.google.com/file/d/1R6HOBJc2368229_BsWBLV0-SbMQlpeIl/view?usp=sharing

In [1]:
import os
import torch
from math import floor
from PIL import Image
from glob import glob
import numpy as np
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.utils.rnn import pack_padded_sequence

In [ ]:
#generate filenames from image ids
# for i in ids:
#     name=str(i)
#     l = len(name)
#     filename=''.join(['0'for x in range(12-l)])+name+".jpg"
#     print(filename)

In [2]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = torch.device("cuda" if use_cuda else "cpu")

CUDA is not available.  Training on CPU ...


In [4]:
sample_data = np.array(glob("/Users/inika/Desktop/Shortcuts/mini/VQA personal/Data/train_sample/*"))
print('There are %d total images.' % len(sample_data))

There are 1913 total images.


In [5]:
images = []
ids = []
for f in sample_data:
    images.append(Image.open(f))
    images[-1].load()
    ids.append(int(f.split("/")[-1].split(".")[0]))

In [6]:
print(type(images[0]))
print(len(images),len(ids))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
1913 1913


In [ ]:
from img2vec_pytorch import Img2Vec
from PIL import Image

img2vec = Img2Vec(cuda=False) #Change this for GPU

In [ ]:
batch = 50
n = floor(len(images)/batch)
vectors = img2vec.get_vec(images[:batch])
k = batch
for i in range(n):
    v = img2vec.get_vec(images[k:k+batch])
    vectors = np.concatenate((vectors,v))
    k+=batch

In [ ]:
df = pd.DataFrame(zip(ids,vectors),columns=['Image ID','Image Vector'])
print(df.head())
df.to_csv('image_vectors_with_ids.csv',index=False)